** PROJECT OBJECTIVE:** Design a python based interactive semi - rule based chatbot which can do the following:

Import Modules

In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import json
import pickle
import random

1. Start chat session with greetings and ask what the user is looking for. 

In [ ]:
def chat():
    print("Hi i am Virtual Assitisnt will help you on Great lakes Education ")
    print("If you want to Quit please endter Quit ")

    while True:
        inp = input("\n\nYou: ")
        if inp.lower() == 'quit':
            break

    #Porbability of correct response 
        results = model.predict([bag_of_words(inp, words)])

    # Picking the greatest number from probability
        results_index = np.argmax(results)

        tag = labels[results_index]

        for tg in data['intents']:

            if tg['tag'] == tag:
                responses = tg['responses']
                print("Bot:\t" + random.choice(responses))

2. Accept dynamic text based questions from the user. Reply back with relevant answer from the designed corpus. 

In [ ]:
#Loading intents.json
with open('GL Bot.json') as intents:
  data = json.load(intents)

stemmer = LancasterStemmer()

# getting informations from intents.json--
words = []
labels = []
x_docs = []
y_docs = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        x_docs.append(wrds)
        y_docs.append(intent['tag'])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])

In [ ]:
# Stemming the words and removing duplicate elements.
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

In [ ]:
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

# One hot encoding, Converting the words to numerals
for x, doc in enumerate(x_docs):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)


    output_row = out_empty[:]
    output_row[labels.index(y_docs[x])] = 1

    training.append(bag)
    output.append(output_row)


training = np.array(training)
output = np.array(output)

In [ ]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=500, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 7999  | total loss: 0.00346 | time: 0.080s
| Adam | epoch: 500 | loss: 0.00346 - acc: 0.9998 -- iter: 120/128
Training Step: 8000  | total loss: 0.00316 | time: 0.085s
| Adam | epoch: 500 | loss: 0.00316 - acc: 0.9998 -- iter: 128/128
--


In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for s_word in s_words:
        for i, w in enumerate(words):
            if w == s_word:
                bag[i] = 1

    return np.array(bag)

3. End the chat session only if the user requests to end else ask what the user is looking for. Loop continues till the user asks to end it.

In [ ]:
chat()

Hi i am Virtual Assitisnt will help you on Great lakes Education 
If you want to Quit please endter Quit 


You: aiml
Bot:	Hello! how can i help you ?


You: AIML
Bot:	Hello! how can i help you ?


You: ML
Bot:	Link: Machine Learning wiki 


You: name please
Bot:	I am your virtual learning assistant


You: quit
